In [ ]:
import numpy as np
import cv2
from collections import defaultdict
from scipy.spatial import KDTree

# Color splitting utility

In [2]:
def get_hue_masks(full_img, reference_HSV, weights = [1.0,0.0,0.0], threshold=0.5, sigma=0.05, sat_min=0.2, val_min=0.2):
	'''
	Gets an image loaded in HSV format and an array of reference HSV colors
	Both are normalized to 1.0 max np.float32 values
	Returns an array of masks, one for each reference HSV color, with white pixels in regions matching
	to a certain degree the corresponding refenence HSV color.
	
	reference_HSV is not supposed to contain black or white, handled by setting minimum values for both saturation (S) and value (V)
	'''
	h, w, _ = full_img.shape
	pixels_hsv = full_img.reshape(-1, 3)  # (num_pixels, 3)

	# Compute circular hue distance
	h_diff = np.abs(pixels_hsv[:, None, 0] - reference_HSV[None, :, 0])
	h_dist = np.minimum(h_diff, 1.0 - h_diff)

	# Mask out low-S or low-V pixels
	low_sv_mask = (pixels_hsv[:,1] < sat_min) | (pixels_hsv[:,2] < val_min)
	h_dist[low_sv_mask, :] = np.inf

	s_dist = np.abs(pixels_hsv[:, None, 1] - reference_HSV[None, :, 1]) if weights[1] > 0 else np.zeros_like(h_dist)
	v_dist = np.abs(pixels_hsv[:, None, 2] - reference_HSV[None, :, 2]) if weights[2] > 0 else np.zeros_like(h_dist)

	w_h, w_s, w_v = weights
	dists = np.sqrt(
		w_h * h_dist**2 
		+ w_s * s_dist**2
		+ w_v * v_dist**2
				 )

	# Soft assignment using Gaussian kernel
	weights_soft = np.exp(- (dists**2) / (2 * sigma**2))  # shape (num_pixels, num_colors)

	# For masked out pixels (inf), set weights to 0
	weights_soft[np.isinf(dists)] = 0

	# Reshape back to (h, w, num_colors)
	weights_img = weights_soft.reshape(h, w, -1)

	# # Plot masks using simple threshold
	# fig, axs = plt.subplots(1, len(reference_HSV), figsize=(22, 4))
	# fig.suptitle("HSV Soft Gaussian weights masks for image test")

	results_hsv = []
	for i in range(len(reference_HSV)):
		channel = weights_img[:, :, i]
		binary_mask = (channel > threshold).astype(np.uint8)
		results_hsv.append(binary_mask * 255)

		# axs[i].imshow(binary_mask, cmap='gray')
		# axs[i].set_title(f"HSV Color {i}")
		# axs[i].axis('off')

	# plt.show()
	return results_hsv

# Line-based ROI detection

In [ ]:
def line_intersection(line1, line2):
	"""
	Finds the intersection of two lines. Each line is defined by two points.
	Returns the intersection point (x, y) or None if lines are parallel.
	"""
	(x1, y1), (x2, y2) = line1
	(x3, y3), (x4, y4) = line2

	den = (x1 - x2) * (y3 - y4) - (y1 - y2) * (x3 - x4)
	if abs(den) < 1e-6:  # Use a small tolerance for floating point comparison
		return None  # Lines are parallel or collinear

	t_num = (x1 - x3) * (y3 - y4) - (y1 - y3) * (x3 - x4)
	t = t_num / den
	
	intersect_x = x1 + t * (x2 - x1)
	intersect_y = y1 + t * (y2 - y1)
	
	return (intersect_x, intersect_y)

def get_internal_angle(p1, p2, p3):
	"""Calculates the internal angle at vertex p2, formed by p1-p2-p3."""
	v1 = np.subtract(p1, p2)
	v2 = np.subtract(p3, p2)

	# suppress warning for division by zero in case of zero-length vectors
	if np.linalg.norm(v1) == 0 or np.linalg.norm(v2) == 0:
		return 0.0
	
	cos_angle = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
	
	# Clip for numerical stability and convert to degrees
	angle = np.degrees(np.arccos(np.clip(cos_angle, -1.0, 1.0)))
	return angle

def is_reasonable_quad(ordered_points, min_angle=45, max_angle=135):
	"""
	Checks if a quadrilateral, defined by ordered points, has reasonable
	internal angles.
	"""
	angles = []
	num_points = len(ordered_points)
	if num_points != 4: 
		return False

	for i in range(num_points):
		p_prev = ordered_points[(i - 1 + num_points) % num_points]
		p_curr = ordered_points[i]
		p_next = ordered_points[(i + 1) % num_points]
		angle = get_internal_angle(p_prev, p_curr, p_next)
		angles.append(angle)
	
	return all(min_angle <= a <= max_angle for a in angles)

def find_loops(segments):
	"""
	Greedily searches for 4-line cycles, returning them as ordered paths.
	"""

	# Build KDTree for fast proximity search
	points = [pt for seg in segments for pt in seg]
	kdtree = KDTree(points)

	# Build fast adjacency list based on endpoint proximity
	adjacency = defaultdict(set)
	for i, (p1, p2) in enumerate(segments):
		for p in [p1, p2]:
			# Find indices of points within a radius 'r'
			proximal_point_indices = kdtree.query_ball_point(np.array(p).reshape(1, -1), r=5.0)[0]
			for j in proximal_point_indices:
				# Map point index back to line index
				other_line_idx = j // 2
				if other_line_idx != i:
					adjacency[i].add(other_line_idx)

	loops = []
	seen_loops = set()  # Tracks unique loops (represented as frozensets)
	for a in adjacency:
		for b in adjacency[a]:
			for c in adjacency[b]:
				if c == a: continue
				# We need a path a->b->c->d->a. A 3-cycle (a->b->c->a) is skipped.
				if a in adjacency[c]: continue
				for d in adjacency[c]:
					if d == a or d == b: continue
					if a in adjacency[d]:  # This closes the 4-line loop
						loop = (a, b, c, d)
						# Canonical representation to detect duplicates
						canonical_loop = frozenset(loop)
						if canonical_loop not in seen_loops:
							loops.append(loop)
							seen_loops.add(canonical_loop)
	return loops

def create_mask_from_loops(loops, segments, image_shape):
	"""
	Processes ordered loops to create a binary mask of valid quadrilaterals.
	"""
	mask = np.zeros(image_shape[:2], dtype=np.uint8)
	valid_rectangle_lines = set()
	
	for loop_indices in loops:  # e.g., (a, b, c, d)
		lines = [segments[i] for i in loop_indices]
		
		# 1. Find the 4 intersection points (vertices) from the ordered path
		v1 = line_intersection(lines[0], lines[1]) # Intersection of a & b
		v2 = line_intersection(lines[1], lines[2]) # Intersection of b & c
		v3 = line_intersection(lines[2], lines[3]) # Intersection of c & d
		v4 = line_intersection(lines[3], lines[0]) # Intersection of d & a
		
		vertices = [v for v in [v1, v2, v3, v4] if v is not None]
		
		# We need exactly 4 vertices to form a quadrilateral
		if len(vertices) != 4:
			continue
			
		# The vertices are already ordered by path traversal.
		ordered_vertices = np.array(vertices, dtype=np.int32)
		
		# 2. Validate the quad's shape using its internal angles
		if is_reasonable_quad(ordered_vertices):
			# 3. If valid, draw it on the mask and save the lines
			cv2.fillPoly(mask, [ordered_vertices], 1)
			valid_rectangle_lines.update(loop_indices)
			
	return mask, list(valid_rectangle_lines)

def get_segments(image):
	# LSD Line Segment Detector
	lsd = cv2.line_descriptor.LSDDetector.createLSDDetector()
	keylines_raw = lsd.detect(image, scale=2, numOctaves=4)

	if keylines_raw is None or len(keylines_raw) < 4:
		return np.zeros(image.shape[:2], dtype=np.uint8)

	min_line_length = 10
	segments = []

	for line in keylines_raw:
		p1 = np.array([line.startPointX, line.startPointY])
		p2 = np.array([line.endPointX, line.endPointY])
		length = np.linalg.norm(p1 - p2)
		
		if length > min_line_length:
			segments.append(((p1[0], p1[1]), (p2[0], p2[1])))
	
	return segments

def get_closed_paths_mask(image):
	"""
	Main function for closed paths detection. Returns a binary mask.
	"""

	segments = get_segments(image)

	if len(segments) < 4:
		return np.zeros(image.shape[:2], dtype=np.uint8)
	
	# Find ordered loops of 4 lines
	loops = find_loops(segments)
	
	# Process loops to create a mask
	mask, _ = create_mask_from_loops(loops, segments, image.shape)
	
	return mask